In [ ]:
import pandas as pd
import seaborn as sns
from google.colab import drive

## Statystyki ubóstwa

Dane pochodzą ze strony [SDG Tracker](https://sdg-tracker.org/no-poverty), która należy do [Our World in Data](https://ourworldindata.org/).

Inicjatywa [SDG Tracker](https://sdg-tracker.org/no-poverty) powstała aby publikować informacje na tematów Celów Zrównoważonego
Rozwoju ONZ.

To cele globalnego rozwoju przyjęte przez ONZ
we wrześniu 2015 roku, które powinny zostać osiągnięte do 2030 roku.

Zapoznajmy się z materiałem dostępnym w notebooku i na podstawie instrukcji przygotujmy dane, które będą potrzebne podczas zajęć.

**Uwaga.** Wracając do dokumentu, pamiętaj o ponownym wywołaniu komórek z kodem.

### Zawartość zbioru danych

W przypadku tego zbioru danych w zestawieniu znajdziemy informacje o:
*   kraju, dla którego podane są statystyki - **Country**,
*   trzyliterowego kodu danego kraju - **Code**,
*   kontynentu - **Continent**,
*   roku, którego dotyczy statystyka - **Year**,
*   udziału populacji żyjącego w skrajnym ubóstwie - **Share Extreme Poverty**,
*   krajowego wskaźnika ubóstwa - **Poverty Headcount Ratio**,
*   wielowymiarowego wskaźnika ubóstwa - **Share Multipoverty**,
*   udziału populacji objętej programami ubezpieczeń socjalnych - **SIP Coverage**,
*   liczby osób zmarłych, zaginionych i bezpośrednio dotkniętych skutkami naturalnych katastrof w przeliczeniu na 100 000 mieszkańców - **Disasters Deaths and Missings**,
*   liczby osób pozbawionych schronienia w wyniku naturalnych katastrof, którzy nie opuścili granic kraju - **Disasters Internally Displaced**,
*   liczby osób bezpośrednio dotkniętych skutkami naturalnych katastrof w przeliczeniu na 100 000 mieszkańców - **Disasters Directly Affected**,
*   wartości strat w wyniku naturalnych katastrof w ujęciu procentowym w odniesieniu do PKB - **Disasters Direct Economic Loss**,
*   udziału wydatków na zdrowie w wydatkach rządowych ogółem - **Gov Expenditure Health**,
*   udziału wydatków na edukację w wydatkach rządowych ogółem - **Gov Expenditure Health**.

Dane zostały wcześniej przygotowane w taki sposób, by połączyć te informacje dla poszczególnych krajów i lat i połączone z informacją o kontynencie.

Potrzebne nam dane zostały udostępnione wraz z dokumentem - poniższy kod importuje plik.

In [ ]:
drive.mount('/content/drive')

In [ ]:
poverty_data = pd.read_csv('/content/drive/My Drive/Vis/Poverty Data/poverty_statistics.csv')
poverty_data

## Przygotowanie zeszytu

Na początku potrzebujemy upewnić się, że dane zostały poprawnie rozpoznane i dokonać potrzebych konwersji.

Przyglądając się danym w podglądzie wyżej, możemy spodziewać się, że informacja o kontynencie będzie ciekawą daną kategoryczną.

W przypadku poszczególnych państw, spodziewamy się znacznej liczby różnych wartości.

W przypadku danych numerycznych, potrzebujemy określić, jak często brakuje informacji w zestawie danych, warto również poznać podstawowe informacje statystyczne.

### Sprawdzenie typów danych

Poniżej potrzebujemy wykonać dołączone instrukcje, aby otrzymać ramki danych wykorzystywane na zajęciach.

W pierwszej kolejności sprawdźmy, jak zostały rozpoznane dane w poszczególnych kolumnach podczas importu.

In [ ]:
poverty_data.dtypes

Określmy również, ile spośród wszystkich rekordów posiada kompletne informacje.

In [ ]:
"complete records: " + str(len(poverty_data.dropna(how="any"))) + "; total records: " + str(len(poverty_data))

W tej sytuacji zastanówmy się, w przypadku jakich kolumn posiadamy najwięcej informacji.

In [ ]:
poverty_data.isnull().sum()

W związku ze sposobem łączenia danych, dla wszystkich rekordów posiadamy informacje o kraju, kodzie, kontynencie i roku.

W przypadku pozostałych statystyk najwięcej danych posiadamy w kolumnie **Share Extreme Poverty**.
Większość rekordów nie posiada informacji w kolumnach **Share Multipoverty** i **SIP Coverage**.

Następne w kolejności są **Poverty Headcount Ratio** i **Disasters Direct Economic Loss**.

W przypadku tych czterech kolumn możemy przypuszczać, że badanie relacji między cechami może być utrudnione - postaramy się w tej sytuacji ustalić zestaw rekordów, w przypadku których jest to uzasadnione.

### Identyfikacja państw w zbiorze

Zastanówmy się, ilu i jakich państw dotyczy zestawienie.

In [ ]:
poverty_data["Country"].value_counts()

Posiadamy statystyki dla 213 państw, przy czym dane są zróżnicowane ze względu na konkretne kraje. Mamy państwa, dla których posiadamy pewne dane dla 41 różnych lat, i takie, dla których posiadamy pewne informacje wyłącznie w przypadku jednego roku.

### Konwersja nazw kontynentów na kategorie

W przypadku kontynentów mamy pewność, że jest ich stosunkowo niewiele. Możemy niemal natychmiast zamienić w tym przypadku typ danych na kategorie.

In [ ]:
poverty_data["Continent"].value_counts()

In [ ]:
poverty_data["Continent"] = poverty_data["Continent"].astype("category")

## Przygotowanie ramek danych

#### Analiza wskaźnika programów socjalnych w stosunku do wydatków państwa

Wybierzmy teraz jeden rok, w przypadku którego obserwujemy względnie niewiele brakujących informacji w wybranych kolumnach.

In [ ]:
years_and_nulls = pd.DataFrame(pd.Series(poverty_data["Year"].unique(), name="unique year"))
years_and_nulls["nulls"] = years_and_nulls["unique year"].apply(lambda x: poverty_data[(poverty_data["Year"]==x)].isnull().sum().sum())
years_and_nulls["records"] = years_and_nulls["unique year"].apply(lambda x: len(poverty_data[(poverty_data["Year"]==x)]))
years_and_nulls["nulls per record"] = years_and_nulls["nulls"]/years_and_nulls["records"]
years_and_nulls.sort_values(by="unique year")
years_and_nulls[years_and_nulls["nulls per record"]==years_and_nulls["nulls per record"].min()]

Wygląda na to, że warto bliżej przyjrzeć się danym dotyczącym roku 2012.

In [ ]:
poverty_data_one_year =  poverty_data[poverty_data["Year"]==2012]
poverty_data_one_year

W tym przypadku posiadamy pewne informacje dla 198 krajów. Zastanówmy się, jakich informacji najczęściej brakuje.

In [ ]:
poverty_data_one_year.isnull().sum()

Dalej wybierzmy rekordy dotyczące 2012 roku, w przypadku których posiadamy informację o **SIP Coverage**, **Share Extreme Poverty** i **Gov Expenditure Education**.

In [ ]:
selected = poverty_data[(poverty_data["Year"]==2012)& (poverty_data["Share Extreme Poverty"].isnull()==False) &
                        (poverty_data["SIP Coverage"].isnull()==False) & (poverty_data["Gov Expenditure Education"].isnull()==False)
]
SIP_extreme_poverty = selected[["Country","Continent","SIP Coverage", "Share Extreme Poverty","Gov Expenditure Education"]]
SIP_extreme_poverty

#### Udział skrajnego ubóstwa w Afryce i Południowej Ameryce

Aby stworzyć porównanie dla dwóch kontynentów ze względu na **Share Extreme Poverty**, potrzebujemy skorzystać z odpowiedniego filtrowania.

Ponieważ kontynent jest rozpoznawany jako kategoria, potrzebujemy również usunąć nieinteresujące nas w tym przypadku kategorie.

In [ ]:
share_extreme_poverty =  poverty_data[
                                      (poverty_data["Share Extreme Poverty"].isnull()==False) 
                                       & ((poverty_data["Continent"]=="Africa")  | (poverty_data["Continent"]=="South America"))
                                      ][["Country",	"Continent", "Year", "Share Extreme Poverty"]]
share_extreme_poverty["Continent"] = share_extreme_poverty["Continent"].cat.remove_unused_categories()
share_extreme_poverty

### Statystyki wybranych kontynentów

Podobnie jak w powyższym przypadku, wybieramy cztery kontynenty, dla których będziemy potrzebowali stworzyć wizualizacje na zajęciach.

Jednocześnie upewniamy się, że dosowaliśmy zakres kategorii, który powinien zostać wyświetlony na wykresie.

In [ ]:
selected_continents =  poverty_data[(poverty_data["Continent"]=="Africa")  
| (poverty_data["Continent"]=="Europe")
| (poverty_data["Continent"]=="Asia")
| (poverty_data["Continent"]=="North America")
                                      ]
selected_continents["Continent"] = selected_continents["Continent"].cat.remove_unused_categories()
selected_continents

## Zadania